# Kinetic library compiler
This script helps compile a kinetic library. By iteratively go through steps, you can build a new kinetic libary with ease.

### Necessary packages

In [ ]:
import logging

from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase
from rmgpy.reaction import Reaction

from toolbox.kineticlib import create_kinetics_entry, \
                               read_kinetic_lib_from_path, \
                               remove_kinetic_entries_from_lib
from toolbox.species import read_spc_dict_from_path, add_spc_to_spc_dict,\
                            write_spc_dict_to_path
from toolbox.reaction import parse_rxn_label

%load_ext autoreload
%autoreload 2

logger = logging.getLogger()
logger.setLevel(logging.INFO)

### [OPTIONAL] Assign a log file to record all the changes 

In [ ]:
fh = logging.FileHandler('kinetic_lib_compiler.log', mode="a+")
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s: %(message)s'))
logger.addHandler(fh)

### Load a RMG database instance

In [ ]:
database = RMGDatabase()
database.load(
    path = settings['database.directory'],
    thermo_libraries = [],  # Can add other library if necessary
    kinetics_families = "default",
    reaction_libraries = [],
    kinetics_depositories = ['training'],
)

kinetic_db = database.kinetics

### [INPUT] Assign the file path of the kinetics library
Hint: Need to place species dictionary file 'dictionary.txt' under the same directory to make it valid.
If the library file does not existed, a new library will be created
- `lib_path` (str): the path to the kinetics library file
- `dict_path` (str): the path to the species dictionary file

In [ ]:
lib_path = ""
dict_path =  ""

In [ ]:
read_kinetic_lib_from_path(lib_path, kinetic_db, create=True)
kinetic_lib = kinetic_db.libraries[lib_path]
spc_dict = read_spc_dict_from_path(dict_path)

### [INPUT] Step 1. Add new species entry
Add new species to the species dictionary. The script can check if you are adding duplicate labels or duplicate species
- `spc_label` (str): the species label. The first character should be alphabetic
- `identifier` (str): the identifier of the species. You can use the SMILES, adjacency list, InChI or common names

In [ ]:
spc_label = "tBA"
identifier = """
tert-butyl acetate
"""

In [ ]:
spc_dict = read_spc_dict_from_path(dict_path)
add_spc_to_spc_dict(spc_label, identifier, spc_dict)
write_spc_dict_to_path(spc_dict, dict_path)

### [INPUT] Step 2. Add reaction label
The reaction label. please use `=>` or `<=>` in the reaction label. If any species label is not recoganizable, the script will ask you to correct the label or extend the species dictionary

In [ ]:
label = "H2O2 <=> O2 + H2"

In [ ]:
reactants, products = parse_rxn_label(label, spc_dict, interactive=True, resonance=True)
reaction = Reaction(reactants=reactants, products=products)
print('label: ' + label )
display(reaction)
# Check balance
if not reaction.is_balanced():
    logging.error("The reaction is not balanced. (may use wrong labels)")
else:
    logging.info("The reaction is balanced.")
# Check duplicate
for item in kinetic_lib.entries.values():
    if reaction.is_isomorphic(item.item):
        logging.warn("This reaction has a duplicate reaction (index:{0})".format(item.index))

### [INPUT] Step 3. Kinetic data settings
This related to settings which are less variant (e.g., the same settings may be consistent across the same paper). Don't assign (`level of theory` and `experiment`) or (`literature_index` and `compute_by`) at the same time.

In [ ]:
k_settings = {
    'E_unit': 'kcal/mol',  # energy unit: J/mol, kJ/mol, cal/mol, kcal/mol
    'A_unit': '1/s',  # A factor unit: 
    'P_unit': 'kPa',  # pressure unit: Pa, kPa, atm, torr, 
    'T_unit': 'K',  # temperature unit: K
    
    'Tmin': 500,
    'Tmax': 2000,
    'Pmin': None,
    'Pmax': None,
    
    'level_of_theory': 'CBS-QB3',
    'experiment': '',
    'literature_index': '1',
    'compute_by': '',
}

### [INPUT] Step 4. Reaction kinetic data
Currently support `Arrhenius`, `MultiArrhenius` and `PdepArrhenius` kinetics types. All Type support `'active'`, `'A'`, `'n'`, `'Ea'`, `'uncertainty'`, `'multiplier'` and `'T0'` as keys. Assign `True` to the key `'active'` to indicate which data and type to be added. For `Arrhenius`, you need to input numbers, while for others, you need to input a list of numbers. 

In [ ]:
k_data = {
 'Arrhenius':
    {'active': False, 'A': 1, 'n': 1, 'Ea': 1, 'uncertainty': None, 'multiplier': 1},
 'MultiArrhenius':
    {'active': False, 
     'A': [1,2],
     'n': [0,0], 
     'Ea': [2,3],
    },
 'PdepArrhenius':
    {'active': True,
     'P': [10, 20],
     'A': [1,2],
     'n': [0,0], 
     'Ea': [2,3],
    }
}

In [ ]:
entry = create_kinetics_entry(label, reaction, len(kinetic_lib.entries), k_data, k_settings, )
print('Index:', entry.index)
print('Kinetics:\n', entry.data)
print('Short desc:\n', entry.short_desc)

### Step 5. Save the species dictionary and library

In [ ]:
kinetic_lib.entries.update({entry.index: entry})
kinetic_lib.save(lib_path)
write_spc_dict_to_path(spc_dict, dict_path)

### Remove reactions
To remove reactions from the library, you need to supply a list of reaction labels

In [ ]:
remove_rxn_label_list = ["H2O2 <=> O2 + H2",]
remove_kinetic_entries_from_lib(remove_rxn_label_list, kinetic_lib)

Save if you are certain that you want to remove these reactions

In [ ]:
kinetic_lib.save(lib_path)